In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import math

In [2]:
class ArcFace(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.clamp(cosine**2, 0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1,1), 1.0)

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


In [3]:
class ResNetArcModel(nn.Module):
    def __init__(self, num_classes, backbone="resnet50", embedding_size=512):
        super(ResNetArcModel, self).__init__()
        resnet = getattr(models, backbone)(weights=True)
        in_features = resnet.fc.in_features
        resnet.fc = nn.Identity()

        self.backbone = resnet
        self.embedding = nn.Linear(in_features, embedding_size)
        self.arcface = ArcFace(embedding_size, num_classes)

    def forward(self, x, labels=None):
        x = self.backbone(x)
        x = self.embedding(x)
        if labels is not None:
            logits = self.arcface(x, labels)
            return logits
        return x


In [5]:
transform = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = datasets.ImageFolder(
    root="../data/preprocessed/train",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

num_classes = len(train_dataset.classes)
print("Clases:", train_dataset.classes)
print("Número de clases:", num_classes)

Clases: ['Abir Ahmed', 'Adriana Sanchez', 'Adriana Solanilla', 'Alejandro Tulipano', 'Amy Olivares', 'Andrea Pipino', 'Blas de Leon', 'Carlos Beitia', 'Carlos Hernandez', 'Cesar Rodriguez', 'Javier Bustamante', 'Jeremy Sanchez', 'Jonathan Peralta', 'Kevin Rodriguez', 'Lucia Cardenas', 'Mahir Arcia', 'Michael Jordan']
Número de clases: 17


In [8]:
model = ResNetArcModel(num_classes=num_classes, embedding_size=512).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
from tqdm import tqdm
import torch

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in pbar:
        images = images.cuda()
        labels = labels.cuda()

        logits = model(images, labels)  # solo si usas ArcFace
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (torch.argmax(logits, dim=1) == labels).sum().item()
        total += labels.size(0)

        acc = correct / total
        pbar.set_postfix(loss=loss.item(), acc=acc)

    avg_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f"✅ Epoch {epoch+1} completed. Avg Loss: {avg_loss:.4f}, Acc: {epoch_acc:.4f}")

# Guardar modelo completo
torch.save(model.state_dict(), "../models/arcface17.pth")

# Guardar solo backbone y embedding
torch.save(
    {
        "backbone": model.backbone.state_dict(),
        "embedding": model.embedding.state_dict()
    },
    "../models/arcface_backbone17.pth"
)


Epoch 1/10: 100%|██████████| 78/78 [00:06<00:00, 12.80it/s, acc=0.998, loss=8.89e-5] 


✅ Epoch 1 completed. Avg Loss: 0.0076, Acc: 0.9976


Epoch 2/10: 100%|██████████| 78/78 [00:06<00:00, 12.79it/s, acc=1, loss=2.91e-5] 


✅ Epoch 2 completed. Avg Loss: 0.0002, Acc: 1.0000


Epoch 3/10: 100%|██████████| 78/78 [00:06<00:00, 12.70it/s, acc=1, loss=0.000881]


...

✅ Epoch 8 completed. Avg Loss: 0.0617, Acc: 0.9948


Epoch 9/10: 100%|██████████| 78/78 [00:06<00:00, 12.20it/s, acc=0.98, loss=0.0035]   


✅ Epoch 9 completed. Avg Loss: 0.2171, Acc: 0.9803


Epoch 10/10: 100%|██████████| 78/78 [00:06<00:00, 12.17it/s, acc=0.973, loss=0.0899]  


✅ Epoch 10 completed. Avg Loss: 0.2434, Acc: 0.9735


In [21]:
import os
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATASET_DIR = "../data/preprocessed/test"

# Cargar backbone
backbone = models.resnet50(weights=None)
in_features = backbone.fc.in_features
backbone.fc = nn.Identity()
checkpoint = torch.load("../models/arcface_backbone17.pth")
backbone.load_state_dict(checkpoint["backbone"])

embedding_layer = torch.nn.Linear(in_features, 512)
embedding_layer.load_state_dict(checkpoint["embedding"])

backbone = backbone.to(DEVICE).eval()
embedding_layer = embedding_layer.to(DEVICE).eval()

transform = transforms.Compose([
    transforms.Resize((112,112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

class_embeddings = {}

for class_name in os.listdir(DATASET_DIR):
    class_dir = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_dir):
        continue

    embeddings = []
    image_files = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".png"))]

    print(f"Procesando {class_name}...")

    for img_file in tqdm(image_files):
        img_path = os.path.join(class_dir, img_file)
        img = Image.open(img_path).convert("RGB")
        x = transform(img).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            features = backbone(x)
            emb = embedding_layer(features)
            emb = F.normalize(emb, dim=1)
        embeddings.append(emb.squeeze(0).cpu())

    mean_emb = torch.stack(embeddings).mean(0)
    mean_emb = F.normalize(mean_emb, dim=0)
    class_embeddings[class_name] = mean_emb.numpy()

np.save("../models/gallery_embeddings17.npy", class_embeddings)
print("✅ Galería guardada en ../models/gallery_embeddings17.npy")

Procesando Abir Ahmed...


100%|██████████| 37/37 [00:00<00:00, 177.60it/s]


Procesando Adriana Sanchez...


100%|██████████| 35/35 [00:00<00:00, 142.17it/s]


Procesando Adriana Solanilla...


...

Procesando Mahir Arcia...


100%|██████████| 39/39 [00:00<00:00, 142.18it/s]


Procesando Michael Jordan...


100%|██████████| 32/32 [00:00<00:00, 147.49it/s]

✅ Galería guardada en ../models/gallery_embeddings17.npy


## Generar TSV
* A partir del gallery_embeddings.npy

Esto con el proposito de poder utilizar la pagina: https://projector.tensorflow.org/

In [22]:
import numpy as np

reference_db = np.load("../models/gallery_embeddings17.npy", allow_pickle=True).item()

# Ordenar por nombre
items = sorted(reference_db.items())

# Embeddings y etiquetas
embeddings = [v for k,v in items]
labels = [k for k,v in items]

# Convertir a array
embeddings = np.stack(embeddings)

# Guardar embeddings.tsv
np.savetxt("../models/embeddings.tsv", embeddings, delimiter="\t", fmt="%.6f")

# Guardar metadata.tsv
with open("../models/metadata.tsv", "w") as f:
    f.write("Label\n")
    for label in labels:
        f.write(f"{label}\n")

print("✅ Archivos embeddings.tsv y metadata.tsv generados.")


✅ Archivos embeddings.tsv y metadata.tsv generados.
